# Chatting with the Knowledge Graph



### Script - intro to chatting

In this notebook, you will be exploring the knowledge graph a bit more.

First with cypher queries to directly explore the graph.

Then using langchain to create a question and answer chat.

Finally, you will use the LLM to combine those techniques.

## Setup

In [ ]:
%run 'shared.ipynb'

# Cypher - queries about addresses


In [ ]:
# Tell me about a manager named royal bank
gdb.execute_query("""
  CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank") YIELD node, score
  RETURN node.name, score LIMIT 1
""").records

In [ ]:
# What is the location of royal bank?
gdb.execute_query("""
CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank"
  ) YIELD node, score
WITH node as mgr LIMIT 1
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr.name, addr
""").records

In [ ]:
# Which state has the most investment firms?
kg.query("""
  MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
  RETURN address.state as state, count(address.state) as numManagers
    ORDER BY numManagers DESC
    LIMIT 10
""")

In [ ]:
# Which state has the most public companies listed?
kg.query("""
  MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
  RETURN address.state as state, count(address.state) as numCompanies
    ORDER BY numCompanies DESC
""")

In [ ]:
# What are the cities in California with the most investment firms?
kg.query("""
  MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
         WHERE address.state = 'California'
  RETURN address.city as city, count(address.city) as numManagers
    ORDER BY numManagers DESC
    LIMIT 10
""")

In [ ]:
# Which city in California has the most companies listed?
kg.query("""
  MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
         WHERE address.state = 'California'
  RETURN address.city as city, count(address.city) as numCompanies
    ORDER BY numCompanies DESC
""")

In [ ]:
# What are top investment firms in San Francisco?
kg.query("""
  MATCH p=(mgr:Manager)-[:LOCATED_AT]->(address:Address),
         (mgr)-[owns:OWNS_STOCK_IN]->(:Company)
         WHERE address.city = "San Francisco"
  RETURN mgr.managerName as city, sum(owns.value) as totalInvestmentValue
    ORDER BY totalInvestmentValue DESC
    LIMIT 10
""")

In [ ]:
# What companies are in Santa Clara?
kg.query("""
  MATCH (com:Company)-[:LOCATED_AT]->(address:Address)
         WHERE address.city = "Santa Clara"
  RETURN com.companyName
""")

### Script - cypher queries with cartesian distance

So far you've been exploring the graph using explicit relationships.

You can also find things based on their location coordinates.

This is like doing vector search, but within a 2-dimensional space

and using cartesian distance rather than cosine similarity.

But the idea is the same.

In [ ]:
# What companies are near Santa Clara?
kg.query("""
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (com:Company)-[:LOCATED_AT]->(companyAddress:Address)
    WHERE point.distance(address.location, companyAddress.location) < 10000
  RETURN com.companyName, com.companyAddress
""")

In [ ]:
# What investment firms are near Santa Clara?
kg.query("""
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress
""")

In [ ]:
# Which investment firms are near Palo Aalto Networks?
kg.query("""
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Aalto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:LOCATED_AT]->(comAddress:Address),
    (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE point.distance(comAddress.location, mgrAddress.location) < 20000
  RETURN mgr, 
    toInteger(point.distance(comAddress.location, mgrAddress.location) / 2000) as distanceKm
    ORDER BY distanceKm ASC
    LIMIT 10
""")

### Script - pause and try out variations

This is a good time to pause the video and try out some variations
of those Cypher queries.

Try different distances, company names and cities to see what you get.

## Cypher - few shot learning

Now to try something new, you can use a few shot learning technique.

Previously, you had been using vector search by itself, or in combination
with a cypher query to provide enough context to answer a user question.

Another approach is to teach the LLM about the knowledge graph,
then ask it to generate Cypher queries for you.

This is called few shot learning. You give the LLM a few examples
of natural language questions and their corresponding Cypher queries.

Then you can ask it to generate a Cypher query for a new question.

### Script - cypher generation template

Let's look at the prompt you will use to teach the LLM about the knowledge graph.

In [ ]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What are the top investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What companies are in Santa Clara?
MATCH (com:Company)-[:LOCATED_AT]->(comAddress:Address)
    WHERE comAddress.city = 'Santa Clara'
RETURN com.companyName

# What investment firms are near Santa Clara?
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, managerAddress.location) < 20 * 1000
  RETURN mgr.managerName, mgr.managerAddress

# Which investment firms are near Palo Aalto Networks?
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Aalto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:LOCATED_AT]->(comAddress:Address),
    (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE point.distance(comAddress.location, mgrAddress.location) < 20 * 1000
  RETURN mgr, 
    toInteger(point.distance(comAddress.location, mgrAddress.location) / 1000) as distanceKm
    ORDER BY distanceKm ASC
    LIMIT 10
  
The question is:
{question}"""

In [ ]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

def prettyCypherChain(question: str) -> str:
    response = cypherChain.run(question)
    print(textwrap.fill(response, 60))


In [ ]:
prettyCypherChain("What investment firms are in San Francisco?")

In [ ]:
prettyCypherChain("What companies are in Santa Clara?")

In [ ]:
prettyCypherChain("What investment firms are near Santa Clara?")

In [ ]:
prettyCypherChain("Which investment firms are near Palo Aalto Networks?")